In [1]:
from citipy import citipy
from config import weather_api_key
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time

In [2]:
### Generate Random Cities

In [3]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

In [4]:
# add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [5]:
# create a list for holding cities
cities = []

# identify the nearest city for each latitude and longitude combination
for c in coordinates: 
    lat, lng = c[0], c[1]
    city = citipy.nearest_city(lat, lng).city_name
    
    if city not in cities:
        # if the name is unique, then add to the cities list
        cities.append(city)

# print the city count to confirm sufficient count
len(cities)

613

In [6]:
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + weather_api_key

### Initialize an Empty List and Counters
### Loop through the List of Cities and Build the City URL

In [7]:
# create an empty list to hold the weather data
city_data = []

# print the beginning of the loutput_data_fileprint('Beginning data retrieval    ')
print('----------------------------')

# create counters
record_count = 1
set_count = 1

# loop through all cities in our list
for i, city in enumerate(cities):
    # group cities in sets of 50 for logging purposes
    if i%50 == 0 and i >= 50:
        set_count += 1
        record_count = 1
        time.sleep(60)
        print('pausing...')
        print('----------------------------\n')
    
    # create endpoint url with each city
    city_url = url + '&q=' + city.replace(' ', '+')
    
    # log the url, record and set numbers and the city
    print(f'Processing record {record_count} of set {set_count} | {city}.')
    record_count += 1

    try: 
        # parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        
        # append the information to the city_data list
        city_data.append(
            {
                'City': city.title(),
                'Lat': city_lat,
                'Lng': city_lng,
                'Max Temp': city_max_temp,
                'Humidity': city_humidity,
                'Cloudiness': city_clouds,
                'Wind Speed': city_wind,
                'Country': city_country,
                'Date': city_date
            }
        )
    except:
        print('City not found. Skipping...')
        pass

# indicate that data loading is complete
print('----------------------------')
print('Data retrieval complete     ')
print('----------------------------')

Beginning data retrieval    
----------------------------
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=arlit
Processing record 1 of set 1 | arlit.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=barrow
Processing record 2 of set 1 | barrow.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=rikitea
Processing record 3 of set 1 | rikitea.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=new+norfolk
Processing record 4 of set 1 | new norfolk.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=provideniya
Processing record 5 of set 1 | provideniya.
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=90abc6c64ffe74abd9fad2385f0f3a29&q=flers
Processing record 6 of set 1 | flers.
http://api.openweathermap.org/

In [ ]:
### Save the `city_data` List as a Pandas DataFrame; Create an output csv file

In [ ]:
# convert the list of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

In [ ]:
column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df = city_data_df[column_order]
city_data_df.head(10)

In [ ]:
# create the output csv file
output_data_file = 'weather_data/cities.csv'
city_data_df.to_csv(output_data_file, index_label='City_ID')